In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
import random

In [42]:
df = pd.read_csv('songdata.csv')

In [43]:
df.head(10)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...
5,ABBA,Burning My Bridges,/a/abba/burning+my+bridges_20003011.html,"Well, you hoot and you holler and you make me ..."
6,ABBA,Cassandra,/a/abba/cassandra_20002811.html,Down in the street they're all singing and sho...
7,ABBA,Chiquitita,/a/abba/chiquitita_20002978.html,"Chiquitita, tell me what's wrong \nYou're enc..."
8,ABBA,Crazy World,/a/abba/crazy+world_20003013.html,I was out with the morning sun \nCouldn't sle...
9,ABBA,Crying Over You,/a/abba/crying+over+you_20177611.html,I'm waitin' for you baby \nI'm sitting all al...


In [44]:
df = df.drop(['link'], axis=1)

In [45]:
df = df.loc[df['artist'] == 'The Beatles']

In [46]:
df.head(5)

,artist,song,text
1198,The Beatles,A Shot Of Rhythm And Blues,"Well, if your hands start a-clappin' \nAnd yo..."
1199,The Beatles,Across The Universe,Words are flowing out like \nEndless rain int...
1200,The Beatles,All I've Got To Do,"Whenever I want you around, yeah \nAll I gott..."
1201,The Beatles,And I Love Her,I give her all my love \nThat's all I do \nA...
1202,The Beatles,And Your Bird Can Sing,You tell me that you've got everything you wan...


In [47]:
df = df.drop(['artist'], axis=1)
df = df.drop(['song'], axis=1)
df.head(5)

,text
1198,"Well, if your hands start a-clappin' \nAnd yo..."
1199,Words are flowing out like \nEndless rain int...
1200,"Whenever I want you around, yeah \nAll I gott..."
1201,I give her all my love \nThat's all I do \nA...
1202,You tell me that you've got everything you wan...


In [115]:
df = df.reset_index(drop=True)
dfTemp = df
lyrics = ''
for i in dfTemp.text:
    lyrics = lyrics + i

In [116]:
for c in ['.',',','?',';','!','\'','\n']:
    lyrics = lyrics.replace(c, '')

words = lyrics.lower().split()
print(random.sample(words, 10))
len(words)

['find', 'da', 'loves', 'i', 'never', 'its', 'aint', 'we', 'keeps', 'friend']


30672

In [139]:
def createMarkovChain(words, prev = None):
    markovChain = dict()
    total = len(words)
    for w in words:
        if w not in markovChain:
            markovChain[w] = [dict(), 0]
        if prev:
            if w not in markovChain[prev][0]:
                markovChain[prev][0][w] = 0
            
            markovChain[prev][0][w] = markovChain[prev][0][w] + 1
            markovChain[prev][1] = markovChain[prev][1] + 1
        prev = w
        
    for w in markovChain:
        succWords = markovChain[w][1]
        markovChain[w] = [markovChain[w][0], markovChain[w][1] / total]
        for s in markovChain[w][0]:
            markovChain[w][0][s] = markovChain[w][0][s] / succWords
    return markovChain

In [140]:
mc = createMarkovChain(words)

In [141]:
cnt = 10
for k in mc:
    print('Words: \'%s\'\n' % k, 'Next:', mc[k])
    cnt = cnt - 1
    if cnt <= 0:
        break

Words: 'well'
 Next: [{'if': 0.050314465408805034, 'with': 0.018867924528301886, 'heres': 0.050314465408805034, 'ive': 0.031446540880503145, 'its': 0.012578616352201259, 'be': 0.018867924528301886, 'you': 0.11949685534591195, 'blue': 0.006289308176100629, 'i': 0.16352201257861634, 'come': 0.012578616352201259, 'well': 0.07547169811320754, 'your': 0.012578616352201259, 'hell': 0.012578616352201259, 'stop': 0.006289308176100629, 'we': 0.012578616352201259, 'do': 0.012578616352201259, 'shes': 0.012578616352201259, 'that': 0.012578616352201259, 'mrdog': 0.012578616352201259, 'have': 0.012578616352201259, 'im': 0.006289308176100629, 'lucille': 0.006289308176100629, 'make': 0.012578616352201259, 'ill': 0.012578616352201259, 'see': 0.018867924528301886, 'look': 0.012578616352201259, 'dont': 0.018867924528301886, 'the': 0.012578616352201259, 'he': 0.012578616352201259, 'this': 0.006289308176100629, 'mamas': 0.006289308176100629, 'she': 0.012578616352201259, 'now': 0.012578616352201259, 'thank'

In [142]:
def createSong(markovChain, length = 240, start = None):
    song = ''
    curr = start
    for cnt in range(length):
        if not curr:
            r = random.random()
            csum = 0.0
            for w in markovChain:
                curr = w
                csum = csum + markovChain[w][1]
                if (r < csum):
                    break
            song = song + ' ' + curr
        r = random.random()
        csum = 0.0
        for succ in markovChain[curr][0]:
            newWords = succ
            csum = csum + markovChain[curr][0][succ]
            if (r < csum):
                break
        song = song + ' ' + succ
        curr = succ
    return song

In [145]:
s = createSong(mc)
s

' a feeling you leave me tonight di-di-di-dindi falling and get in all you go dont let you go soon miss lizzy girl girl and start my feet back get back my baby youre thinking of course henry the brand new year evrybody put their foot dont ya know he please come up roll up to you dr robertyour day not always be my life would rather small they said goodbye hello hello" i got a dog till i needed someone to me just for every way that you know how they said i aint she acts as i love love you was a thing to i want me cry over your hands) off my) when you feel glad its lonely you every single day sunshine good night on me she really sorry that love with a falling and im nobodys childi read the middle of it seems that is up she keeps calling me love with your mother mary comes near a birthday well your fears you can you came just the boys now all together now hes a loser and its just a number but im doing the end through the ocean my woman way and have a little girl who came here id like to se